# Model Usage

Bridgic provides a concise and rich encapsulation for model usage. Including:

- Support for model integration with multiple model providers: `OpenAI LLM`, `OpenAI Like LLM`, `Vllm Server LLM`.
- Support multiple methods of model invocation: `chat`, `stream`, `structured output`.

The information passed to the model during these model invocation processes are all `Message` objects in Bridgic.The return results of model invocation vary depending on the invocation method used.

## Model Integration

In the actual production process, many models may be used, and these models come from different providers. For ease of using, Bridgic also provides corresponding usage methods.

### OpenAI LLM

Common usage for OpenAI model. Initialize an instance of an OpenAI model using `OpenAILlm` for invocation.

In [ ]:
# Get the environment variables.
import os
from dotenv import load_dotenv
load_dotenv()

from bridgic.core.model.types import Message, Role
from bridgic.llms.openai.openai_llm import OpenAILlm

# Set the OpenAI Key
_api_key = os.environ.get("OPENAI_API_KEY")
_model_name = "gpt-4.1-mini"

# Initialize the OpenAI Llm.
llm = OpenAILlm(api_key=_api_key, timeout=10)

# Chat.
response = llm.chat(
    model=_model_name,
    messages=[
        Message.from_text(text="You are a helpful assistant.", role=Role.SYSTEM),
        Message.from_text(text="Hello!", role=Role.USER),
    ]
)
print(response.message.content)

Hello! How can I assist you today?


### OpenAI Like LLM

Some LLM providers, not OpenAI's models, offer interfaces compatible with OpenAI models. Initialize an instance using `OpenAILikeLlm` for invocation.

In [ ]:
# Get the environment variables.
import os
from dotenv import load_dotenv
load_dotenv()

from Bridgic.core.intelligence import Message, Role
from Bridgic.llms.openai_like.openai_like_llm import OpenAILikeLlm

# Set the API base and key.
_api_base = os.environ.get("OPENAI_LIKE_API_BASE")
_api_key = os.environ.get("OPENAI_LIKE_API_KEY")
_model_name = os.environ.get("OPENAI_LIKE_MODEL_NAME")

# Initialize the OpenAI Llm.
llm = OpenAILikeLlm(api_base=_api_base, api_key=_api_key, timeout=10)

# Chat.
response = llm.chat(
    model=_model_name,
    messages=[
        Message.from_text(text="You are a helpful assistant.", role=Role.SYSTEM),
        Message.from_text(text="Hello!", role=Role.USER),
    ]
)
print(response.message.content)

Hello! How can I assist you today?


### Vllm Server LLM



In [ ]:
# Get the environment variables.
import os
from dotenv import load_dotenv
load_dotenv()

from Bridgic.core.intelligence import Message, Role
from Bridgic.llms.vllm.vllm_server_llm import VllmServerLlm

# Set the API base and key.
_api_base = os.environ.get("VLLM_SERVER_API_BASE")
_api_key = os.environ.get("VLLM_SERVER_API_KEY")
_model_name = os.environ.get("VLLM_SERVER_MODEL_NAME")

# Initialize the OpenAI Llm.
llm = VllmServerLlm(api_base=_api_base, api_key=_api_key, timeout=10)

# Chat.
response = llm.chat(
    model=_model_name,
    messages=[
        Message.from_text(text="You are a helpful assistant.", role=Role.SYSTEM),
        Message.from_text(text="Hello!", role=Role.USER),
    ]
)
print(response.message.content)

Hello! How can I assist you today? 😊


## Model Invocation

When invoking the model, depending on the different scenarios, we may expect it to have different behaviors. For instance, in a chatbot, we expect it to return a stream, while in a fixed pipeline scenario, we expect it to return a fixed data structure. Bridgic has designed relevant calling methods for these needs.

> Not all invocation methods can be executed for sure. Some invocation methods can run correctly only when the model supports the relevant protocols at the same time.

Let's take VllmServerLlm as an example to introduce each calling method.

In [ ]:
# Get the environment variables.
import os
from dotenv import load_dotenv
load_dotenv()

# Import the necessary modules.
from Bridgic.core.intelligence import Message, Role, Response, StreamResponse, AsyncStreamResponse
from Bridgic.llms.vllm.vllm_server_llm import VllmServerLlm

# Set the API base and key.
_api_base = os.environ.get("VLLM_SERVER_API_BASE")
_api_key = os.environ.get("VLLM_SERVER_API_KEY")
_model_name = os.environ.get("VLLM_SERVER_MODEL_NAME")

# Initialize the OpenAI Llm.
vllm_llm = VllmServerLlm(api_base=_api_base, api_key=_api_key, timeout=10)

### Chat

`chat()` and `achat()` directly invoke the model and return a `Response` object, which encapsulates the model's output.

In [11]:
response: Response = vllm_llm.chat(
    model=_model_name,
    messages=[
        Message.from_text(text="You are a helpful assistant.", role=Role.SYSTEM),
        Message.from_text(text="Hello!", role=Role.USER),
    ]
)
print(f"response: {response}")
print(f"content: {response.message.content}")

response: message=Message(role=<Role.AI: 'assistant'>, blocks=[TextBlock(block_type='text', text='Hello! 😊 How can I assist you today?')], extras={}) raw=ChatCompletion(id='chatcmpl-92b13da2cd0347aaa90f2171eb424843', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! 😊 How can I assist you today?', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[], reasoning_content=None), stop_reason=None)], created=1760338633, model='Qwen/Qwen3-4B-Instruct-2507', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=12, prompt_tokens=21, total_tokens=33, completion_tokens_details=None, prompt_tokens_details=None), prompt_logprobs=None, kv_transfer_params=None)
content: Hello! 😊 How can I assist you today?


### Stream

`stream()` return a generator `StreamResponse` objects, which encapsulates the model's streaming output. Each chunk of the stream is encapsulated in a `MessageChunk` object. `astream()` returns an asynchronous generator `AsyncStreamResponse` object.

In [ ]:
response: StreamResponse = vllm_llm.stream(
    model=_model_name,
    messages=[
        Message.from_text(text="You are a helpful assistant.", role=Role.SYSTEM),
        Message.from_text(text="Describe yourself with two words.", role=Role.USER),
    ]
)
print(type(response))
print(f"------use the stream------")
for chunk in response:
    print(chunk.delta)
print(f"------end of the stream------\n")

async_response: AsyncStreamResponse = vllm_llm.astream(
    model=_model_name,
    messages=[
        Message.from_text(text="You are a helpful assistant.", role=Role.SYSTEM),
        Message.from_text(text="Describe yourself with two words.", role=Role.USER),
    ]
)
print(type(async_response))
print(f"------use the async stream------")
async for chunk in async_response:
    print(chunk.delta)
print(f"------end of the async stream------\n")

<class 'generator'>
------use the stream------

Help
ful
,
 thoughtful

------end of the stream------

<class 'async_generator'>
------use the async stream------

Help
ful
,
 thoughtful
.

------end of the async stream------



### Structured Output



`structured_output()` and `astructured_output()` are used to invoke the model to return the output in the specified format. The format can be pydantic basemodel, JSON schema, regular expression, EBNF grammar, etc. 

> Note: The prerequisite for this method to operate correctly is that the model service used supports the corresponding structured output. For instance, models deployed through Vllm support a relatively wide variety of structured outputs: [vLLM Structured Output](https://docs.vllm.ai/en/latest/features/structured_outputs.html) and OpenAI natively only supports pydantic BaseModel and some JSON schemas: [OpenAI Structured Output](https://platform.openai.com/docs/guides/structured-outputs)

For example, invoke the model to extract names from a given text and return the `Names`.

In [ ]:
from pydantic import BaseModel, Field
from typing import List
from Bridgic.core.intelligence import PydanticModel

class Names(BaseModel):
    names: List[str] = Field(description="The names extracted from the text.")

response: Names = vllm_llm.structured_output(
    model=_model_name,
    messages=[
        Message.from_text(text="Given a text, extract the names from it.", role=Role.SYSTEM),
        Message.from_text(text="The text is: 'John Doe, Jane Smith, and Bob Johnson.'", role=Role.USER),
    ],
    constraint=PydanticModel(model=Names)
)
print(type(response))
print(response)

<class '__main__.Names'>
names=['John Doe', 'Jane Smith', 'Bob Johnson']


Or, we can also make it return a JSON schema.

In [ ]:
from typing import Dict
from Bridgic.core.intelligence import JsonSchema

schema = {
    "type": "object",
    "properties": {
        "names": {"type": "array", "description": "The names extracted from the text.", "items": {"type": "string"}},
    },
    "required": ["names"],
}
response: Dict = vllm_llm.structured_output(
    model=_model_name,
    messages=[
        Message.from_text(text="Given a text, extract the names from it.", role=Role.SYSTEM),
        Message.from_text(text="The text is: 'John Doe, Jane Smith, and Bob Johnson.'", role=Role.USER),
    ],
    constraint=JsonSchema(name="ExtractNames", schema_dict=schema)
)
print(type(response))
print(response)

<class 'dict'>
{'names': ['John Doe', 'Jane Smith', 'Bob Johnson']}


Structured output also supports outputting strings described by the given regular expression.

In [ ]:
from Bridgic.core.intelligence import Regex

response: str = vllm_llm.structured_output(
    model=_model_name,
    messages=[
        Message.from_text(text="You are a helpful assistant.", role=Role.SYSTEM),
        Message.from_text(text="This year is 2025. Please tell me the date of this year's Christmas.", role=Role.USER),
    ],
    constraint=Regex(pattern=r"\d{4}-\d{2}-\d{2}", description="The date should be in the format of YYYY-MM-DD.")
)
print(response)

2025-12-25
